# Color quantization

Para reducir el tamaño de una imagen podemos elegir los colores más frecuentes y representarlos mediante una etiqueta entera. Los colores típicos se pueden encontrar mediante el algoritmo *K-means*. Para decidir la etiqueta de cada pixel hacemos una búsqueda de vecino más próximo (*nearest neighbour*).

Adaptado del [ejemplo de scikit-learn](http://scikit-learn.org/stable/auto_examples/cluster/plot_color_quantization.html).

In [ ]:
%matplotlib inline

import numpy             as np
import cv2               as cv
import matplotlib.pyplot as plt
import glob
from matplotlib.pyplot   import imshow, subplot, title, plot
from sklearn.cluster import KMeans
from sklearn.utils import shuffle


def readrgb(file):
    return cv.cvtColor( cv.imread(file), cv.COLOR_BGR2RGB) 

def fig(w,h):
    return plt.figure(figsize=(w,h))

def imshowg(x):
    imshow(x, 'gray')

In [ ]:
def flat(img):
    h,w,d = img.shape
    return np.reshape(img, [w*h, d]).astype(float)/255

def codebook(n, imgs, m=10000):
    pixels = np.vstack([flat(x) for x in imgs])
    sample = shuffle(pixels, random_state=0)[:m]
    code = KMeans(n_clusters=n, n_init='auto').fit(sample)
    return code

def encode(code,x):
    h,w,d = x.shape
    return np.reshape(code.predict(flat(x)),[h,w])

def recreate(code,l):
    c = code.cluster_centers_
    return c[l,:]

In [ ]:
imgs = [cv.resize(readrgb(f),(0,0), fx=1/4, fy=1/4) for f in glob.glob('../images/naranjas/*.jpg') ]

In [ ]:
# después puedes cambiar de imagen
x = imgs[1]

print(x.dtype)
imshow(x);
print(flat(x).dtype)
print(np.max(flat(x)))

In [ ]:
# número de colores
code = codebook(10,imgs)

labs = encode(code,x)

rec  = recreate(code,labs)

fig(12,4)
subplot(1,2,1); imshow(labs)
subplot(1,2,2); imshow(rec);

In [ ]:
fig(16,8)
subplot(1,2,1); imshow((rec[200:300,300:400])) #*255).astype(np.uint8));
subplot(1,2,2); imshow(  x[200:300,300:400]);

In [ ]:
code.cluster_centers_

In [ ]:
pal = np.outer(np.ones([10]),sorted(code.cluster_centers_,key=lambda c: -c[1])).reshape(10,code.n_clusters,3)
imshow(pal,interpolation='none');

In [ ]:
imshow(code.transform(code.cluster_centers_),cmap='gray',interpolation='none');

## k-d tree

El tiempo de cómputo del vecino más próximo se puede reducir precomputando un [k-dtree](https://en.wikipedia.org/wiki/K-d_tree). En este [blog post](http://blog.krum.io/k-d-trees/) se muestran algunos experimentos.

En algunas aplicaciones se utiliza una búsqueda aproximada más rápida del vecino más próximo ([FLANN](https://www.cs.ubc.ca/research/flann/)).